In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# self write tools
from tools import LABEL_ENCODER
from models import RNN

<IPython.core.display.Javascript object>

# Check GPU

In [2]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("Using {}: {}".format(device, torch.cuda.get_device_name(0)))

Using cuda: GeForce RTX 3080


# 1. Global Variables

In [3]:
BATCH_SIZE = 512


TRAIN_CSV = 'input/split/train.csv'
VALID_CSV = 'input/split/valid.csv'


# 2. Preprocess data

## 2.1 Read data

In [4]:
def remov_duplicates(input): 
    input = input.split(" ") 
    for i in range(0, len(input)): 
        input[i] = "".join(input[i]) 
    UniqW = Counter(input) 
    s = " ".join(UniqW.keys()) 
    return s

#tokenization
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

def encode_sentence(text, vocab2index, N=15):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

def preprocess_dataset(path):

    # read csv
    reviews = pd.read_csv(path)

    # combine categories
    # reviews['description'] = reviews['noisyTextDescription']
    reviews['description'] = reviews['gender'] + ' ' + reviews['baseColour'] + ' ' + reviews['usage'] + ' ' + reviews['noisyTextDescription']
    reviews = reviews[['id', 'description', 'category']]

    # remove duplicat
    reviews['description'] = reviews['description'].apply(lambda x: remov_duplicates(x))
    reviews['description_length'] = reviews['description'].apply(lambda x: len(x.split()))

    #count number of occurences of each word
    counts = Counter()
    for index, row in reviews.iterrows():
        counts.update(tokenize(row['description']))
    
    #deleting infrequent words
    print("num_words before:",len(counts.keys()))
    for word in list(counts):
        if counts[word] < 2:
            del counts[word]
    print("num_words after:",len(counts.keys()))

    #creating vocabulary
    vocab2index = {"":0, "UNK":1}
    words = ["", "UNK"]
    for word in counts:
        vocab2index[word] = len(words)
        words.append(word)

    reviews['encoded'] = reviews['description'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))

    return reviews, words
    

In [5]:

reviews, words = preprocess_dataset("input/train.csv")

X = list(reviews['encoded'])
y = list(reviews['category'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=11)

y_train, y_valid = LABEL_ENCODER.transform(y_train), LABEL_ENCODER.transform(y_valid)
y_valid




num_words before: 6968
num_words after: 6614


array([24, 10, 21, ...,  2, 24, 21])

In [6]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [7]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [8]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    best_valid_loss = float('inf')
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)

        # save best model
        if val_loss < best_valid_loss:
            best_valid_loss = val_loss
            torch.save(model.state_dict(), './output/rnn-model.pt')

        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.2f%%, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]

    

    return sum_loss/total, correct/total * 100, sum_rmse/total

In [9]:
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE)

# Model

# Train

In [10]:
model = RNN(vocab_size, 50, 50)

In [11]:
train_model(model, epochs=100, lr=0.001)

train loss 2.197, val loss 1.926, val accuracy 48.06%, and val rmse 9.682
train loss 1.106, val loss 0.962, val accuracy 75.38%, and val rmse 6.534
train loss 0.760, val loss 0.709, val accuracy 81.62%, and val rmse 5.507
train loss 0.586, val loss 0.584, val accuracy 84.95%, and val rmse 5.061
train loss 0.482, val loss 0.529, val accuracy 86.06%, and val rmse 4.928
train loss 0.416, val loss 0.493, val accuracy 87.01%, and val rmse 4.712
train loss 0.364, val loss 0.477, val accuracy 87.56%, and val rmse 4.724
train loss 0.315, val loss 0.456, val accuracy 87.63%, and val rmse 4.802
train loss 0.279, val loss 0.447, val accuracy 88.33%, and val rmse 4.617
train loss 0.242, val loss 0.438, val accuracy 88.16%, and val rmse 4.679
train loss 0.219, val loss 0.436, val accuracy 88.63%, and val rmse 4.596
train loss 0.198, val loss 0.444, val accuracy 88.63%, and val rmse 4.606
train loss 0.179, val loss 0.434, val accuracy 88.74%, and val rmse 4.562
train loss 0.162, val loss 0.439, val 

# Analyze results

In [12]:
model.load_state_dict(torch.load('output/rnn-model.pt'))

<All keys matched successfully>